# 🚀 Pharma-Safe Lens: Kaggle Backend Deployment

## Phase 6: Deploy FastAPI Backend on Kaggle with ngrok

This notebook deploys the complete FastAPI backend on Kaggle and exposes it via ngrok tunnel, allowing your local React frontend to connect to it.

### Architecture:
```
[Local Machine: React Frontend] 
         ↓ HTTP Requests
[ngrok Tunnel] 
         ↓
[Kaggle: FastAPI Backend + MedGemma]
```

### What This Does:
1. ✅ Installs system dependencies (Tesseract OCR)
2. ✅ Clones your GitHub repository
3. ✅ Installs Python requirements
4. ✅ Configures ngrok tunneling
5. ✅ Starts FastAPI backend on port 8000
6. ✅ Exposes backend publicly via ngrok
7. ✅ Provides URL for frontend configuration

### Requirements:
- ⚡ Kaggle Notebook Settings:
  - **Internet: ON** (required for git clone & ngrok)
  - **Accelerator: GPU T4 x2** (REQUIRED for MedGemma model)
  - **Persistence: Session Only**

- 🔑 ngrok Account:
  - Create free account at https://ngrok.com
  - Get auth token from https://dashboard.ngrok.com/get-started/your-authtoken
  - Add token to Kaggle Secrets as `NGROK_AUTH_TOKEN`

- 🤗 HuggingFace Access (for MedGemma):
  - Accept license at https://huggingface.co/google/medgemma-4b-it
  - Get access token from https://huggingface.co/settings/tokens
  - Add token to Kaggle Secrets as `HF_TOKEN`

---

Let's begin! 🎯

## Step 1: Install System Dependencies

In [ ]:
# Install Tesseract OCR (required for Phase 1)
!apt-get update -y
!apt-get install -y tesseract-ocr

# Verify installation
!tesseract --version

print("\n✅ System dependencies installed successfully!")

## Step 2: Clone GitHub Repository

⚠️ **IMPORTANT**: Replace `YOUR_USERNAME` with your actual GitHub username!

In [ ]:
# Clone repository
# REPLACE YOUR_USERNAME with your GitHub username!
!git clone https://github.com/YOUR_USERNAME/pharma-safe-lens.git

# Navigate to project
%cd pharma-safe-lens

# Verify structure
!ls -la

print("\n✅ Repository cloned successfully!")

## Step 3: Install Python Dependencies

In [ ]:
# Navigate to backend directory
%cd backend

# Install all requirements
!pip install -r requirements.txt

# Install ngrok
!pip install pyngrok

print("\n✅ All dependencies installed!")
print("⏱️ This takes ~3-5 minutes")

## Step 4: Verify Installation & Imports

In [ ]:
# Add project to path
import sys
sys.path.insert(0, '/kaggle/working/pharma-safe-lens')

# Test imports
from backend.app.main import app
from backend.app.drug_db import DrugDatabase
from backend.app.interaction_logic import InteractionChecker
from backend.app.inference import AIInference
from backend.app.safety import SafetyGuard

print("✅ All backend modules imported successfully!")
print("✅ FastAPI app loaded!")
print(f"✅ {len(DrugDatabase().drugs)} drugs in database")
print(f"✅ {len(InteractionChecker().interactions)} interactions in database")

## Step 5: Configure ngrok

### Option A: Use Kaggle Secrets (Recommended)
1. Go to Kaggle Account Settings → API
2. Add Secret: `NGROK_AUTH_TOKEN` = `your_token_here`
3. In notebook settings, add the secret

### Option B: Paste Token Directly
Replace `YOUR_NGROK_TOKEN` in the cell below with your actual token.

In [ ]:
from pyngrok import ngrok
import os

# Try to get token from Kaggle Secrets first
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    NGROK_TOKEN = user_secrets.get_secret("NGROK_AUTH_TOKEN")
    print("✅ Using ngrok token from Kaggle Secrets")
except:
    # Fallback: paste token directly
    NGROK_TOKEN = "YOUR_NGROK_TOKEN"  # Replace with your token!
    print("⚠️ Using hardcoded ngrok token")

# Set auth token
ngrok.set_auth_token(NGROK_TOKEN)

print("✅ ngrok configured successfully!")

## Step 6: Configure HuggingFace Token (Required for MedGemma)

### 🔐 MedGemma is a Gated Model
The `google/medgemma-4b-it` model requires HuggingFace authentication.

### Prerequisites:
1. ✅ Accept license at: https://huggingface.co/google/medgemma-4b-it
2. ✅ Get token from: https://huggingface.co/settings/tokens

### Add Token to Kaggle Secrets:
1. Click **Add-ons** → **Secrets** in notebook sidebar
2. Add new secret: **Name:** `HF_TOKEN`, **Value:** `your_token_here`
3. Enable the secret checkbox

Run the cell below to verify token access:

In [ ]:
import os

# Configure HuggingFace token for gated model access
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
    
    if HF_TOKEN:
        os.environ['HF_TOKEN'] = HF_TOKEN
        os.environ['HUGGING_FACE_HUB_TOKEN'] = HF_TOKEN
        print("✅ HuggingFace token configured from Kaggle Secrets")
        print(f"   Token: {HF_TOKEN[:8]}...{HF_TOKEN[-4:]}")
    else:
        print("⚠️ HF_TOKEN secret exists but is empty")
        print("   Please set a valid token in Kaggle Secrets")
except Exception as e:
    print(f"⚠️ Could not load HF_TOKEN from Kaggle Secrets: {e}")
    print("   The model will use mock inference as fallback")
    print("")
    print("📝 To enable real MedGemma:")
    print("   1. Click 'Add-ons' → 'Secrets' in sidebar")
    print("   2. Add secret: Name='HF_TOKEN', Value=your_token")
    print("   3. Enable the checkbox next to HF_TOKEN")
    print("   4. Re-run this cell")

## Step 7: Start FastAPI Backend & Create ngrok Tunnel

This will:
1. Start FastAPI server on port 8000
2. Load MedGemma model (if HF_TOKEN configured)
3. Create ngrok tunnel
4. Display public URL
5. Keep server running

⚠️ **The server will run indefinitely. Do not stop this cell!**

In [ ]:
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import time

# Apply nest_asyncio to allow uvicorn in Jupyter
nest_asyncio.apply()

# Create ngrok tunnel BEFORE starting uvicorn
print("🔧 Creating ngrok tunnel...")
public_url = ngrok.connect(8000)
print("\n" + "="*70)
print("🎉 BACKEND IS READY!")
print("="*70)
print(f"\n🌐 Public URL: {public_url}")
print(f"\n📋 Frontend Configuration:")
print(f"   1. Copy the URL above")
print(f"   2. Open: frontend/src/services/api.js")
print(f"   3. Replace API_BASE_URL with: '{public_url}'")
print(f"   4. Save and restart frontend: npm run dev")
print("\n" + "="*70)
print("\n⚠️  IMPORTANT: Keep this cell running!")
print("⚠️  If you stop it, the backend will shut down.")
print("⚠️  Maximum runtime: 12 hours on Kaggle")
print("\n🔄 Starting FastAPI server...")
print("="*70 + "\n")

# Start uvicorn server
try:
    uvicorn.run(
        app,
        host="0.0.0.0",
        port=8000,
        log_level="info"
    )
except KeyboardInterrupt:
    print("\n🛑 Server stopped by user")
    ngrok.kill()
except Exception as e:
    print(f"\n❌ Server error: {e}")
    ngrok.kill()

## 🎯 Next Steps

### 1. Update Frontend API URL
```javascript
// frontend/src/services/api.js
const API_BASE_URL = 'https://YOUR-NGROK-URL.ngrok.io'; // Use URL from above
```

### 2. Start Frontend Locally
```bash
cd frontend
npm install  # First time only
npm run dev
```

### 3. Test Application
1. Frontend opens at: http://localhost:5173
2. Upload a prescription image
3. Wait 10-20 seconds for analysis
4. View detailed results!

---

##  Troubleshooting

### Backend Issues:
- **Error: "Module not found"** → Re-run Step 3 (install dependencies)
- **Error: "Address already in use"** → Restart kernel and run again
- **ngrok error** → Check your auth token is correct
- **CORS error** → Verify URL in frontend matches ngrok URL exactly

### Frontend Issues:
- **Network error** → Check ngrok URL is correct in api.js
- **CORS blocked** → Backend CORS is already configured, check URL
- **Upload fails** → Check file is JPG/PNG and <2MB

### Kaggle Issues:
- **Internet not working** → Enable "Internet" in notebook settings
- **Session timeout** → Kaggle max 12 hours, restart notebook
- **Out of memory** → Enable GPU if needed

---

## 📊 Monitor Backend Logs

The cell above will show live logs including:
- Incoming requests
- OCR results
- Drug detection
- Interaction analysis
- Errors (if any)

Watch the logs to see your frontend requests being processed! 🎉